In [2]:
from abc import ABC, abstractmethod

class LinearCode(ABC):
    
    def __init__(self, code=None):
        self.code = code
    
    def set_code(self, code):
        self.code = code
    
    def get_code(self):
        return self.code
    
    @abstractmethod
    def get_parity_check_matrix(self):
        ...
                
    @staticmethod
    def serialize_instance(self, target_file_name):
        pass
    
    @staticmethod
    def load_from_file(self):
        pass

In [3]:
class GRSCode(LinearCode):
    
    def __init__(self, code=None, n=10, k=5, q=11):
        if code == None:
            self.code = self.generate_code(n, k, q)
        else:
            self.code = code
    
    def get_parity_check_matrix(self):
        return self.code.parity_check_matrix()
    
    def generate_code(self, n, k, q):
        F = GF(q)
        return codes.GeneralizedReedSolomonCode(F.list()[:n], k)

In [4]:
code = GRSCode()
print(code.get_parity_check_matrix())

[10  9  8  7  6  5  4  3  2  1]
[ 0  9  5 10  2  3  2 10  5  9]
[ 0  9 10  8  8  4  1  4  7  4]
[ 0  9  9  2 10  9  6  6  1  3]
[ 0  9  7  6  7  1  3  9  8  5]


In [5]:

class CodeBasedCryptosystem(ABC):
    def __init__(self, code, public_key=None, private_key=None):
        self.code = code
        if public_key == None or private_key == None:
            self.public_key, self.private_key = self.generate_keypair()
        else:
            self.public_key = public_key
            self.private_key = private_key
    
    @abstractmethod
    def generate_keypair(self):
        ...
    
    @abstractmethod
    def encrypt(self, message):
        ...
    
    @abstractmethod
    def decrypt(self, cryptotext):
        ...

In [15]:
from math import floor

class Niederreiter(CodeBasedCryptosystem):
    def generate_random_nonsingular_matrix(self, size):
        S = random_matrix(ZZ, size)
        
        while not S.is_singular() and S.determinant()==0:
            S = random_matrix(ZZ, size)
        
        return S
    
    def generate_random_permutation_matrix(self, n):
        return Permutations(n).random_element().to_matrix()
    
    def encode_(self, n, t):
        enc = [0]*n
        count = t
        while count>0:
            r = randrange(n)
            if enc[r] == 0:
                enc[r]+=1
                count-=1
        return matrix(enc)
    
    def encrypt(self, message=None):
        H_pub, t = self.public_key
        n = H_pub.ncols()
        
        if message == None:
            message = self.encode_(n, t)
        
        print("Message is: "+ str(message))
        
        return matrix(QQ, H_pub) * matrix(QQ, message.transpose())
        
    
    def decrypt(self, cryptotext):
        H, S, P = self.private_key
        #print(cryptotext)
        print("SIZES \n")
        print((~S).nrows(), (~S).ncols())
        
        HPMt = (~S) * cryptotext
        return HPMt
        
    
    def generate_keypair(self):
        H = code.get_parity_check_matrix()
        n, k = H.ncols(), H.nrows()
        S = self.generate_random_nonsingular_matrix(n - k)
        P = self.generate_random_permutation_matrix(n)
        t = floor((n - k) / 2)
        
        return ((S * H * P, t), (H, S, P))

In [16]:
nied = Niederreiter(code=code)
print(nied.public_key)
print(nied.private_key)

([ 2 10  1  0 10  9  6  5  0  1]
[ 2  2  1  6  1  9  3 10  9  1]
[10 10 10  5 10  7  7  6  1  0]
[ 3 10  9  3  3  5  6  1  4  0]
[ 6  6  3  5  9  1  9  6  2  8], 2)
([10  9  8  7  6  5  4  3  2  1]
[ 0  9  5 10  2  3  2 10  5  9]
[ 0  9 10  8  8  4  1  4  7  4]
[ 0  9  9  2 10  9  6  6  1  3]
[ 0  9  7  6  7  1  3  9  8  5], [ 1 -1 -1 -1  2]
[-1 -6 -1 -3  1]
[ 1 -2 -3 -1 -1]
[ 8  3 -1 -1  0]
[ 2 -8  0  6 -1], [0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 1 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0 0 0])


In [17]:
enc = nied.encrypt()
print("Encrypted message is: \n"+str(enc))
dec = nied.decrypt(enc)
print("Decrypted message is: "+str(dec))

Message is: [0 0 0 1 0 0 0 1 0 0]
Encrypted message is: 
[ 5]
[16]
[11]
[ 4]
[11]
SIZES 

5 5
Decrypted message is: [   579/632]
[  -637/316]
[-3803/2528]
[-3069/2528]
[ -411/1264]
